In [4]:
import uuid
from datetime import datetime, timedelta

import redis
import pandas as pd

# Conexión a Redis (contendor en localhost:6379)
r = redis.Redis(
    host="localhost",
    port=6379,
    db=0,
    decode_responses=True,  # para trabajar con strings en vez de bytes
)

# Check rápido
print("PING ->", r.ping())

PING -> True


In [5]:
# Helpers para construir claves, respetando el modelo definido

def notif_hash_key(notif_id: str) -> str:
    return f"notif:{notif_id}"

def user_scheduled_key(user_id: str, ts: int) -> str:
    # timestamp en segundos (epoch)
    return f"notif:{user_id}:scheduled:{ts}"

def user_queue_key(user_id: str) -> str:
    return f"notif:{user_id}:queue"

def user_delivered_key(user_id: str, notif_id: str) -> str:
    return f"notif:{user_id}:delivered:{notif_id}"

def user_prefs_key(user_id: str) -> str:
    return f"notif:preferences:{user_id}"


In [6]:
def create_notification( user_id: str, notif_type: str, payload: dict, send_at: datetime, ttl_seconds: int, status: str = "pending",):
    """
    Crea una notificación siguiendo el modelo:
    - notif/{notifId} (hash) + TTL
    - notif/{userId}/scheduled/{timestamp}
    - notif/{userId}/queue (push)
    """
    notif_id = str(uuid.uuid4())
    notif_key = notif_hash_key(notif_id)

    # Hash de la notificación
    mapping = {
        "notifId": notif_id,
        "userId": user_id,
        "type": notif_type,
        "payload": str(payload),  # para simplificar: str del dict
        "createdAt": datetime.utcnow().isoformat(),
        "sendAt": send_at.isoformat(),
        "status": status,
    }

    # Guardar hash
    r.hset(notif_key, mapping=mapping)

    # TTL sobre la notificación
    r.expire(notif_key, ttl_seconds)

    # Registrar como "programada" por usuario
    ts_epoch = int(send_at.timestamp())
    scheduled_key = user_scheduled_key(user_id, ts_epoch)
    r.set(scheduled_key, notif_id)
    r.expire(scheduled_key, ttl_seconds)

    # Encolar en la queue del usuario (para consumo sencillo)
    r.rpush(user_queue_key(user_id), notif_id)

    return notif_id


def mark_notification_delivered(user_id: str, notif_id: str, seen: bool = False):
    """
    - Actualiza el status del hash notif/{notifId}
    - Crea la clave notif/{userId}/delivered/{notifId} con TTL corto
    """
    notif_key = notif_hash_key(notif_id)

    status = "sent"
    if seen:
        status = "seen"

    # actualizar hash
    r.hset(notif_key, "status", status)
    r.hset(notif_key, "deliveredAt", datetime.utcnow().isoformat())

    delivered_key = user_delivered_key(user_id, notif_id)
    r.set(delivered_key, status)

    # por ejemplo, guardamos la info 7 días
    r.expire(delivered_key, 7 * 24 * 3600)


def set_user_notification_preferences(user_id: str, **prefs):
    """
    Guarda el hash de preferencias de notificación del usuario:
    notif:preferences/{userId}
    Ej: email_enabled=True, push_enabled=False, quiet_hours="22-08"
    """
    pref_key = user_prefs_key(user_id)
    mapping = {k: str(v) for k, v in prefs.items()}
    r.hset(pref_key, mapping=mapping)


In [7]:
# Limpieza opcional de claves previas de este notebook
for key in r.scan_iter("notif:*"):
    r.delete(key)

print("Claves 'notif:*' borradas")

# Preferencias de ejemplo
set_user_notification_preferences(
    "user_1",
    email_enabled=True,
    push_enabled=True,
    quiet_hours="23-07",
)

set_user_notification_preferences(
    "user_2",
    email_enabled=False,
    push_enabled=True,
    quiet_hours="00-00",
)

# Crear algunas notificaciones de ejemplo
now = datetime.utcnow()

notif_ids = []
notif_ids.append(
    create_notification(
        user_id="user_1",
        notif_type="streakReminder",
        payload={"message": "¡No pierdas tu racha de 10 días!", "lang": "es"},
        send_at=now + timedelta(minutes=5),
        ttl_seconds=3600,  # 1 hora
    )
)

notif_ids.append(
    create_notification(
        user_id="user_1",
        notif_type="lessonPending",
        payload={"message": "Te espera una lección de inglés", "lang": "en"},
        send_at=now + timedelta(minutes=30),
        ttl_seconds=7200,  # 2 horas
    )
)

notif_ids.append(
    create_notification(
        user_id="user_2",
        notif_type="achievement",
        payload={"message": "¡Nuevo logro: Unidad 3 completada!", "lang": "es"},
        send_at=now + timedelta(minutes=1),
        ttl_seconds=3600,
    )
)

print("Notificaciones creadas:", notif_ids)


Claves 'notif:*' borradas
Notificaciones creadas: ['8959228c-9689-4952-9f95-9205f66e9e18', '14aeced9-7dd4-499b-8b9e-8af7b74f765e', '9bc56663-cce4-4ccb-97d1-64b512568b72']


C:\Users\Matias Praderi\AppData\Local\Temp\ipykernel_10360\2240126987.py:23: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()
C:\Users\Matias Praderi\AppData\Local\Temp\ipykernel_10360\4038764647.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "createdAt": datetime.utcnow().isoformat(),


In [8]:
def get_user_queue(user_id: str):
    """Devuelve la lista de notifIds en la cola de un usuario."""
    return r.lrange(user_queue_key(user_id), 0, -1)


def get_notifications_details(notif_ids):
    """Devuelve un DataFrame con los hashes de las notificaciones dadas."""
    rows = []
    for nid in notif_ids:
        data = r.hgetall(notif_hash_key(nid))
        if data:
            rows.append(data)
    if not rows:
        return pd.DataFrame()
    return pd.DataFrame(rows)


user = "user_1"

queue_ids = get_user_queue(user)
print("Queue de usuario:", user, "=>", queue_ids)

df_user_notifs = get_notifications_details(queue_ids)
df_user_notifs


Queue de usuario: user_1 => ['8959228c-9689-4952-9f95-9205f66e9e18', '14aeced9-7dd4-499b-8b9e-8af7b74f765e']


,notifId,userId,type,payload,createdAt,sendAt,status
0,8959228c-9689-4952-9f95-9205f66e9e18,user_1,streakReminder,{'message': '¡No pierdas tu racha de 10 días!'...,2025-11-30T18:48:53.746993,2025-11-30T18:53:53.746461,pending
1,14aeced9-7dd4-499b-8b9e-8af7b74f765e,user_1,lessonPending,"{'message': 'Te espera una lección de inglés',...",2025-11-30T18:48:53.750246,2025-11-30T19:18:53.746461,pending


In [9]:
def get_user_scheduled(user_id: str):
    """
    Busca las claves notif:{userId}:scheduled:{timestamp}
    y retorna un DataFrame ordenado por timestamp.
    """
    pattern = f"notif:{user_id}:scheduled:*"
    rows = []
    for key in r.scan_iter(pattern):
        # extraer timestamp de la clave
        parts = key.split(":")
        ts = int(parts[-1])
        notif_id = r.get(key)
        rows.append(
            {
                "key": key,
                "timestamp": ts,
                "sendAt_dt": datetime.utcfromtimestamp(ts),
                "notifId": notif_id,
            }
        )

    if not rows:
        return pd.DataFrame()

    rows = sorted(rows, key=lambda x: x["timestamp"])
    return pd.DataFrame(rows)


get_user_scheduled("user_1")


C:\Users\Matias Praderi\AppData\Local\Temp\ipykernel_10360\1219103228.py:17: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "sendAt_dt": datetime.utcfromtimestamp(ts),


,key,timestamp,sendAt_dt,notifId
0,notif:user_1:scheduled:1764539633,1764539633,2025-11-30 21:53:53,8959228c-9689-4952-9f95-9205f66e9e18
1,notif:user_1:scheduled:1764541133,1764541133,2025-11-30 22:18:53,14aeced9-7dd4-499b-8b9e-8af7b74f765e


In [10]:
# Marcamos como entregada y vista la primera notificación de user_1
if queue_ids:
    first_notif = queue_ids[0]
    mark_notification_delivered("user_1", first_notif, seen=True)

    delivered_key = user_delivered_key("user_1", first_notif)
    print("Clave delivered:", delivered_key)
    print("Valor:", r.get(delivered_key))

    # También ver el hash actualizado
    r.hgetall(notif_hash_key(first_notif))


Clave delivered: notif:user_1:delivered:8959228c-9689-4952-9f95-9205f66e9e18
Valor: seen


C:\Users\Matias Praderi\AppData\Local\Temp\ipykernel_10360\4038764647.py:53: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  r.hset(notif_key, "deliveredAt", datetime.utcnow().isoformat())


In [11]:
def get_user_preferences(user_id: str):
    return r.hgetall(user_prefs_key(user_id))

print("Preferencias user_1:")
print(get_user_preferences("user_1"))

print("\nPreferencias user_2:")
print(get_user_preferences("user_2"))


Preferencias user_1:
{'email_enabled': 'True', 'push_enabled': 'True', 'quiet_hours': '23-07'}

Preferencias user_2:
{'email_enabled': 'False', 'push_enabled': 'True', 'quiet_hours': '00-00'}
